In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
from tqdm import tqdm

In [9]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [16]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("lusxvr/distilbert-base-multilingual-cased-finetuned-leiko")
model = AutoModelForMaskedLM.from_pretrained("lusxvr/distilbert-base-multilingual-cased-finetuned-leiko")

# Specify the patter to use for this lexical simplifier
bert_pattern = '{original_sentence} Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}'

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [17]:
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
bert_ls = SimpleBertLexicalSimplifier(model, tokenizer, bert_pattern, None)

No exemplars provided, using zero-shot mode.


In [ ]:
from benchmark_suite import BenchmarkSuite
from language import Language
suite = BenchmarkSuite(bert_ls, {Language.EN: '{original_sentence} The simpler version of the previous sentence is: {sentence_with_complex_word_masked}',
                                 Language.DE: '{original_sentence} Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}'})
suite.run()

Benchmarking model on EN ...
Benchmarking model on BenchLSDataProvider...


Benchmarking: 100%|██████████| 929/929 [00:11<00:00, 81.78it/s]


Benchmarking model on LexMTurkDataProvider...


Benchmarking:  79%|███████▉  | 397/500 [00:04<00:01, 89.64it/s]